To calculate FID scores for our experiments, we will generate a synthetic dataset using each version of the trained generator. The size of the generated dataset will match that of the original Monet painting dataset for distribution comparison.

In [1]:
# Imports
import os
import torch
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
from tqdm import tqdm

In [2]:
# Generator class
import torch.nn as nn

class ResBlock(nn.Module):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.InstanceNorm2d(channels),
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=0),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
        )

        # ResNet blocks
        self.resnet_blocks = nn.Sequential(*[ResBlock(256) for _ in range(9)])

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, kernel_size=7, stride=1, padding=0),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.resnet_blocks(x)
        x = self.decoder(x)
        return x


In [3]:
transform = transforms.Compose([
    transforms.Resize((256,256), interpolation=Image.BICUBIC),transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
])

In [4]:
# input and output directories
input_path = 'C:/Users/itayg/Documents/MonetGAN/Data/photo_jpg'
output_path = 'C:/Users/itayg/Documents/MonetGAN/Data/monet_generated'

In [5]:
def denormalize(tensor):
    """
    Converts a tensor from [-1, 1] to [0, 1] for display.
    Expects tensor shape (C, H, W) or (B, C, H, W).
    """
    tensor = tensor * 0.5 + 0.5
    return tensor.clamp(0, 1)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
G_A2B = Generator().to(device)
G_A2B.load_state_dict(torch.load('generator_photo2monet_epoch195_identity_1.5.pth'))
G_A2B.eval();

In [ ]:
images = tqdm(os.listdir(input_path)):300

for img in tqdm(images):
    if img.lower().endswith(('.jpg')):
        img_path = os.path.join(input_path, img)

        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output_tensor = G_A2B(input_tensor)

        output_tensor = denormalize(output_tensor.squeeze(0)).clamp(0, 1)
        save_image(output_tensor, os.path.join(output_path, img))

print ('All images were generated and saved in:', output_path)

NameError: name 'img_path' is not defined

In [ ]:
import torch
import matplotlib.pyplot as plt

def show_generated_samples(generator, real_photo, epoch):
    generator.eval()  # Set the generator to evaluation mode
    with torch.no_grad():
        fake_monet = generator(real_photo.to(device))  # Generate Monet-style image

    # Get first image in batch
    real_photo = real_photo[0].cpu().detach()
    fake_monet = fake_monet[0].cpu().detach()

    # De-normalize from [-1, 1] → [0, 1]
    real_photo = denormalize(real_photo).permute(1, 2, 0)
    fake_monet = denormalize(fake_monet).permute(1, 2, 0)

    # Plot images side by side
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[0].imshow(real_photo.numpy())
    ax[0].set_title("Real Photo")
    ax[0].axis("off")

    ax[1].imshow(fake_monet.numpy())
    ax[1].set_title(f"Generated Monet (Epoch {epoch})")
    ax[1].axis("off")

    plt.tight_layout()

In [ ]:
from PIL import Image

test_input_path = 'C:/Users/itayg/Documents/MonetGAN/Data/photo_jpg/dd036adfe5.jpg'
test_input = Image.open(test_input_path).convert('RGB')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5), (0.5))
])
test_input = transform(test_input).unsqueeze(0)
test_input = test_input.to(device)
show_generated_samples(G_A2B, test_input, 4.5)